In [24]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DataFrame").getOrCreate()

OPERATIONS ON DATAFRAME
1. Reading file & Create data frame
2. Checking Schema of dataframe
3. Selecting columns and index
4. Describe() method on dataframe
5. Operation with column (add, drop, rename)
6. Nested schema

1. READ THE CSV FILE WITH PYSPARK

In [2]:
# order_df = spark.read.format("csv").option("header", "true") \
#     .option("inferSchema", "true") \
#     .load("data/student.csv")

In [2]:

order_df = spark.read.csv('data/student.csv', inferSchema=True, header=True)
order_df.show(5)

+---+----------+-----+----+------+
| id|      name|class|mark|gender|
+---+----------+-----+----+------+
|  1|  John Deo| Four|  75|female|
|  2|  Max Ruin|Three|  85|  male|
|  3|    Arnold|Three|  55|  male|
|  4|Krish Star| Four|  60|female|
|  5| John Mike| Four|  60|female|
+---+----------+-----+----+------+
only showing top 5 rows



1B. CREATE A DATAFRAME
- Define structure/schema of dataframe
- Create or load data
- Create dataframe by createDataFrame(data= , schema= )

In [ ]:
demo_data = [
    (1,"Alice","3A",80,"female"),
    (2,"Bob","3A",85,"male"),
    (3,"Cather","3B",90,"female"),
    (4,"Danny","3B",79,"male"),
    (5,"David","3C",68,"male"),
    (6,"Elle","3A",90,"female"),
]

Method 1: Using StructType, StructField,...

In [27]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [35]:
schema_1 = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("class", StringType(), True),
    StructField("mark", IntegerType(), True),
    StructField("gender", StringType(), True),
])

In [36]:
demo_df1 = spark.createDataFrame(data=demo_data, schema=schema_1)

In [37]:
demo_df1.show()

+---+------+-----+----+------+
| id|  name|class|mark|gender|
+---+------+-----+----+------+
|  1| Alice|   3A|  80|female|
|  2|   Bob|   3A|  85|  male|
|  3|Cather|   3B|  90|female|
|  4| Danny|   3B|  79|  male|
|  5| David|   3C|  68|  male|
|  6|  Elle|   3A|  90|female|
+---+------+-----+----+------+



Method 2: Define schema by using string
- "<col_name> <data_type>, <col_name> <data_type>,..."

In [38]:
schema_2 = "id long, name string, class string, mark integer, gender string"

In [39]:
demo_df2= spark.createDataFrame(demo_data, schema_2)

In [40]:
demo_df2.show()

+---+------+-----+----+------+
| id|  name|class|mark|gender|
+---+------+-----+----+------+
|  1| Alice|   3A|  80|female|
|  2|   Bob|   3A|  85|  male|
|  3|Cather|   3B|  90|female|
|  4| Danny|   3B|  79|  male|
|  5| David|   3C|  68|  male|
|  6|  Elle|   3A|  90|female|
+---+------+-----+----+------+



Method 3: Define only list of Column name

In [42]:
schema_3 = ['id', 'name','class','mark','gender']

In [43]:
demo_df3 = spark.createDataFrame(demo_data, schema_3)

Method 4: Convert data from rdd to dataframe
- Note: Must remove the header, unless err will happend

In [93]:
rdd = spark.sparkContext.textFile('data/student_no_header.csv')

In [87]:
type(rdd)
demo = rdd.take(4)[1]

pyspark.rdd.RDD

In [99]:
convert_data = rdd.map(lambda row: ( 
    int(row.split(',')[0]),
    row.split(',')[1],
    row.split(',')[2],
    int(row.split(',')[3]),
    row.split(',')[4]
    ))

In [100]:
convert_data.take(2)

[(1, 'John Deo', 'Four', 75, 'female'), (2, 'Max Ruin', 'Three', 85, 'male')]

In [101]:
demo_df4 = spark.createDataFrame(convert_data, schema_2)

In [102]:
demo_df4.show(5)

+---+----------+-----+----+------+
| id|      name|class|mark|gender|
+---+----------+-----+----+------+
|  1|  John Deo| Four|  75|female|
|  2|  Max Ruin|Three|  85|  male|
|  3|    Arnold|Three|  55|  male|
|  4|Krish Star| Four|  60|female|
|  5| John Mike| Four|  60|female|
+---+----------+-----+----+------+
only showing top 5 rows



In [103]:
demo_df1.printSchema()
demo_df2.printSchema()
demo_df3.printSchema()
demo_df4.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- class: string (nullable = true)
 |-- mark: integer (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- class: string (nullable = true)
 |-- mark: integer (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- class: string (nullable = true)
 |-- mark: long (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- class: string (nullable = true)
 |-- mark: integer (nullable = true)
 |-- gender: string (nullable = true)



2. CHECKING SCHEMA OF DATAFRAME

In [4]:
order_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- class: string (nullable = true)
 |-- mark: integer (nullable = true)
 |-- gender: string (nullable = true)



In [5]:
type(order_df)

pyspark.sql.dataframe.DataFrame

3. Selecting columns and index

In [ ]:
# order_df.head(number_of_rows)
order_df.head(3)

[Row(id=1, name='John Deo', class='Four', mark=75, gender='female'),
 Row(id=2, name='Max Ruin', class='Three', mark=85, gender='male'),
 Row(id=3, name='Arnold', class='Three', mark=55, gender='male')]

In [19]:
# Check type of columns
order_df.dtypes

[('id', 'int'),
 ('name', 'string'),
 ('class', 'string'),
 ('mark', 'int'),
 ('gender', 'string')]

In [ ]:
order_df.select(["id","name"])

DataFrame[id: int, name: string]

In [12]:
order_df.select(["id","name"]).show(3)

+---+--------+
| id|    name|
+---+--------+
|  1|John Deo|
|  2|Max Ruin|
|  3|  Arnold|
+---+--------+
only showing top 3 rows



4. Describe() method on dataframe

- This method includes: count, min, max, stddev, mean and 
- operates on numberic - string values

In [21]:
order_df.describe().show()

+-------+------------------+---------+-----+------------------+------+
|summary|                id|     name|class|              mark|gender|
+-------+------------------+---------+-----+------------------+------+
|  count|                35|       35|   35|                35|    35|
|   mean|              18.0|     NULL| NULL| 74.65714285714286|  NULL|
| stddev|10.246950765959598|     NULL| NULL|16.401116994139826|  NULL|
|    min|                 1|Alex John|Eight|                18|female|
|    max|                35|    Tumyu|Three|                96|  male|
+-------+------------------+---------+-----+------------------+------+



5. Operating columns in data frame (add, drop, rename,...)

In [3]:
order_df.show(3)

+---+--------+-----+----+------+
| id|    name|class|mark|gender|
+---+--------+-----+----+------+
|  1|John Deo| Four|  75|female|
|  2|Max Ruin|Three|  85|  male|
|  3|  Arnold|Three|  55|  male|
+---+--------+-----+----+------+
only showing top 3 rows



In [10]:
adding_df = order_df.withColumn('major', order_df['class'])
adding_df.show(3)

+---+--------+-----+----+------+-----+
| id|    name|class|mark|gender|major|
+---+--------+-----+----+------+-----+
|  1|John Deo| Four|  75|female| Four|
|  2|Max Ruin|Three|  85|  male|Three|
|  3|  Arnold|Three|  55|  male|Three|
+---+--------+-----+----+------+-----+
only showing top 3 rows



In [14]:
adding_df = order_df.withColumns({'major':order_df['class'], 
                                  'total_mark':20+order_df['mark']})
adding_df.show(3)

+---+--------+-----+----+------+-----+----------+
| id|    name|class|mark|gender|major|total_mark|
+---+--------+-----+----+------+-----+----------+
|  1|John Deo| Four|  75|female| Four|        95|
|  2|Max Ruin|Three|  85|  male|Three|       105|
|  3|  Arnold|Three|  55|  male|Three|        75|
+---+--------+-----+----+------+-----+----------+
only showing top 3 rows



In [18]:
drop_df = adding_df.drop('major', 'total_mark')
drop_df.show(3)

+---+--------+-----+----+------+
| id|    name|class|mark|gender|
+---+--------+-----+----+------+
|  1|John Deo| Four|  75|female|
|  2|Max Ruin|Three|  85|  male|
|  3|  Arnold|Three|  55|  male|
+---+--------+-----+----+------+
only showing top 3 rows



In [19]:
rename_df = adding_df.withColumnRenamed('major', 'major_2')
rename_df.show(3)

+---+--------+-----+----+------+-------+----------+
| id|    name|class|mark|gender|major_2|total_mark|
+---+--------+-----+----+------+-------+----------+
|  1|John Deo| Four|  75|female|   Four|        95|
|  2|Max Ruin|Three|  85|  male|  Three|       105|
|  3|  Arnold|Three|  55|  male|  Three|        75|
+---+--------+-----+----+------+-------+----------+
only showing top 3 rows



In [20]:
rename_df = adding_df.withColumnsRenamed({'major':'major_2', 'total_mark':'total'})
rename_df.show(3)

+---+--------+-----+----+------+-------+-----+
| id|    name|class|mark|gender|major_2|total|
+---+--------+-----+----+------+-------+-----+
|  1|John Deo| Four|  75|female|   Four|   95|
|  2|Max Ruin|Three|  85|  male|  Three|  105|
|  3|  Arnold|Three|  55|  male|  Three|   75|
+---+--------+-----+----+------+-------+-----+
only showing top 3 rows



6. Nested schema

In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.appName('Nested schema').getOrCreate()

In [4]:
json_data = [
    {'cust_id': 1, 'full_name': {'first_name': 'Alice', 'last_name': 'DE'}, 'city': 'Washington DC'},
    {'cust_id': 2, 'full_name': {'first_name': 'Alice', 'last_name': 'DE'}, 'city': 'Washington DC'},
    {'cust_id': 3, 'full_name': {'first_name': 'Alice', 'last_name': 'DE'}, 'city': 'Washington DC'},
]

In [3]:
nest_schema = "cust_id long, full_name struct<first_name: string, last_name: string>, city string"

In [8]:
from pyspark.sql.types import *

In [10]:
nest_schema2 = StructType([
    StructField(name='cust_id', dataType=LongType(), nullable=False),
    StructField('full_name', StructType([
        StructField('first_name', StringType(),True), 
        StructField('last_name', StringType(), True)])),
    StructField('city', StringType(), True)
])

In [13]:
nested_df1 = spark.createDataFrame(json_data, nest_schema)

In [12]:
nested_df2 = spark.createDataFrame(json_data, nest_schema2)

In [15]:
nested_df1.show()
nested_df2.show()

+-------+-----------+-------------+
|cust_id|  full_name|         city|
+-------+-----------+-------------+
|      1|{Alice, DE}|Washington DC|
|      2|{Alice, DE}|Washington DC|
|      3|{Alice, DE}|Washington DC|
+-------+-----------+-------------+

+-------+-----------+-------------+
|cust_id|  full_name|         city|
+-------+-----------+-------------+
|      1|{Alice, DE}|Washington DC|
|      2|{Alice, DE}|Washington DC|
|      3|{Alice, DE}|Washington DC|
+-------+-----------+-------------+



In [14]:
nested_df1.printSchema()
nested_df2.printSchema()

root
 |-- cust_id: long (nullable = true)
 |-- full_name: struct (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |-- city: string (nullable = true)

root
 |-- cust_id: long (nullable = false)
 |-- full_name: struct (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |-- city: string (nullable = true)



In [29]:
cust_df = spark.read.format('json').option('multiLine',True).option('header',True).load('data/customer.json')

In [30]:
cust_df.show(truncate=False)

+-------------------+---+----+--------------------------------------------------------------------+
|address            |id |name|phone_numbers                                                       |
+-------------------+---+----+--------------------------------------------------------------------+
|{New York, NY}     |1  |John|[{123-456-7890, NULL, home, NULL}, {NULL, 987-654-3210, NULL, work}]|
|{San Francisco, CA}|2  |Jane|[{555-1234, NULL, mobile, NULL}, {NULL, 777-4321, NULL, work}]      |
+-------------------+---+----+--------------------------------------------------------------------+



Flattening data by explode()

In [20]:
from pyspark.sql.functions import explode

In [31]:
cust_df.columns

['address', 'id', 'name', 'phone_numbers']

In [26]:
flat_df = cust_df.select('id', 'name', explode('phone_numbers').alias('phone'))

In [33]:
flat_df.select('id','name','phone.type','phone.number').show()

+---+----+------+------------+
| id|name|  type|      number|
+---+----+------+------------+
|  1|John|  home|123-456-7890|
|  1|John|  work|987-654-3210|
|  2|Jane|mobile|    555-1234|
|  2|Jane|  work|    777-4321|
+---+----+------+------------+

